In [30]:
import os
import glob
import cv2
import random
from PIL import Image
from tqdm import tqdm

def save_to_frame(input_dir, save_dir, category):
    """
    input:
        input_dir: directory to the video files
        save_dir: directory to save the frames converted from the video
    output:
        None
    """
    os.makedirs(save_dir, exist_ok=True)
    for path in tqdm(glob.glob(input_dir + '/*')):
        fname = os.path.basename(path).split('.')[0]
#         os.makedirs(os.path.join(save_dir, fname), exist_ok=True)
        vidcap = cv2.VideoCapture(path)
        success, image = vidcap.read()
        count = 0
        while success:
            if count % 1 == 0:
                #print("count = {}".format(count))
                cv2.imwrite("{}/{}-{}.jpg".format(save_dir, fname, str(count).zfill(4)), image)     # save frame as JPEG file            
            success, image = vidcap.read()
            count += 1


In [31]:
save_to_frame("./HockeyFights/violence", "./HockeyFightFrames/violence", "violence")

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:22<00:00,  6.04it/s]


In [32]:
save_to_frame("./HockeyFights/nonViolence", "./HockeyFightFrames/nonViolence", "nonViolence")

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:22<00:00,  6.08it/s]


In [2]:
#data preprocessing for feeding into lstm

import cv2
import os


dataset=[]


def load_images_from_folder(folder,x):
    
    image=[]
    im=[]
    c=1
    
    
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        n_img=cv2.resize(img,(100,100))
        if img is not None :
            im.append(n_img)
        if c%10==0:
            image.append(im)
            image.append(x)
            im=[]
            dataset.append(image)
            image=[]
        c=c+1
        
load_images_from_folder("./trial_frames/training/nonViolence",0)
load_images_from_folder("./trial_frames/training/violence",1)

In [3]:
print(len(dataset))
# print(dataset)

import random
train_data = []
test_data = []

368


In [4]:
import random
random.shuffle(dataset)
for samples in dataset[:10]:
    print(samples[1])
x=[]
y=[]
for img,lab in dataset:
    x.append(img)
    y.append(lab)

1
0
0
1
1
0
1
0
0
1


In [5]:
import numpy as np
x=np.array(x).reshape(-1,10,100,100,3)
print(x.shape)

(368, 10, 100, 100, 3)


In [6]:
#saving everything as pickle
import pickle
pickle_out=open("xhockey.pickle","wb")
pickle.dump(x,pickle_out)
pickle_out.close()


pickle_out=open("yhockey.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

In [7]:
#loading from pickle
import pickle
pickle_in=open("xhockey.pickle","rb")
x=pickle.load(pickle_in)
pickle_in.close()

pickle_in=open("yhockey.pickle","rb")
y=pickle.load(pickle_in)
pickle_in.close()

In [8]:
#data preprocessing for feeding into lstm

import cv2
import os


validset=[]


def load_images_from_folder(folder,x):
    
    image=[]
    im=[]
    c=1
    
    
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        n_img=cv2.resize(img,(100,100))
        if img is not None :
            im.append(n_img)
        if c%10==0:
            image.append(im)
            image.append(x)
            im=[]
            validset.append(image)
            image=[]
        c=c+1
        
load_images_from_folder("./trial_frames/testing/nonViolence",0)
load_images_from_folder("./trial_frames/testing/violence",1)

In [9]:
print(len(validset))

40


In [10]:
import random
random.shuffle(validset)
for samples in validset[:10]:
    print(samples[1])
x_valid=[]
y_valid=[]
for img,lab in validset:
    x_valid.append(img)
    y_valid.append(lab)

0
0
0
0
1
0
1
0
0
1


In [12]:
import numpy as np
x_valid=np.array(x_valid).reshape(-1,10,100,100,3)
print(x_valid.shape)

(40, 10, 100, 100, 3)


In [13]:
x_valid=x_valid.astype('float32')/255
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

In [26]:
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TimeDistributed,Dropout, Activation, Flatten,Conv2D, MaxPooling2D,LSTM,Bidirectional
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications import MobileNetV2

# K.image_data_format('tf')
import numpy as np
x=x.astype('float32')/255
NAME="HOCKEYFINAL"
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))

mobilenet = MobileNetV2(
    input_shape=None,
#     alpha=1.0,
    include_top= False,
    weights=None,
    input_tensor=None,
    pooling=None,
#     classes=3,
#     classifier_activation="softmax"
)

## training the CNN
cnn = Sequential()

cnn.add(mobilenet)
#input
# cnn.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

#1st layer

# cnn.add(Conv2D(64, (3, 3), activation='relu'))
# cnn.add(MaxPooling2D((2, 2)))
# cnn.add(Conv2D(64, (3, 3), activation='relu'))
# cnn.add(MaxPooling2D((2, 2)))
# cnn.add(Conv2D(64, (3, 3), activation='relu'))
# cnn.add(MaxPooling2D((2, 2)))

#converting to 1-d tensor
cnn.add(Flatten())

model=Sequential()
model.add(TimeDistributed(cnn,input_shape=x.shape[1:]))
model.add(Bidirectional(LSTM(32)))
#model.add(LSTM(32))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
#model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_3 (TimeDis  (None, 10, 11520)        2257984   
 tributed)                                                       
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               2957568   
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 2)                 66        
                                                                 
Total params: 5,221,858
Trainable params: 5,187,746
No

In [27]:
x = np.array(x)
y = np.array(y)
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)

model.fit(x,y,epochs=25,validation_data=(x_valid,y_valid),batch_size=5, callbacks=[tensorboard])
model.save("Hockey.h5")

Epoch 1/25
74/74 [==============================] - 153s 2s/step - loss: 0.7105 - accuracy: 0.4891 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/25
74/74 [==============================] - 142s 2s/step - loss: 0.6988 - accuracy: 0.4755 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/25
74/74 [==============================] - 141s 2s/step - loss: 0.6967 - accuracy: 0.4891 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/25
74/74 [==============================] - 144s 2s/step - loss: 0.6975 - accuracy: 0.4728 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/25
74/74 [==============================] - 145s 2s/step - loss: 0.6961 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/25
74/74 [==============================] - 142s 2s/step - loss: 0.6962 - accuracy: 0.4783 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/25
74/74 [==============================] - 143s 2s/step - loss: 0.6956 - accuracy: 0.4511 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 8/25
74